Nhiệm vụ 3 - GroupBy & Tổng hợp: 

Mục tiêu : phân nhóm dữ liệu và rút ra các đặc trưng quan trọng.
- Tính điểm trung bình theo vị trí.
- Tính mức lương đề nghị trung bình theo vị trí.
- Xác định ứng viên nổi bật.
- Phân tích tương quan giữa kinh nghiệm và mức lương đề nghị.
- Tổng hợp điểm theo từng ứng viên.


In [14]:
import pandas as pd

# Chuẩn bị dữu liệu
# Load các file dữ liệu đã làm sạch
df_ung_vien = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_candidate_profile.csv')
df_diem = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_interview_score.csv')
df_luong = pd.read_csv('D:/Nhom_5_Case_6/Data_Clean/cleaned_salary_offer.csv')


# Chuẩn hóa cơ bản (Bắt buộc để GroupBy đúng)
for df in [df_ung_vien, df_diem, df_luong]:
    df['candidate_id'] = df['candidate_id'].str.upper().str.strip()

# Chuẩn hóa Vị trí
df_luong['position'] = df_luong['position'].str.strip().str.title()
tu_dien_vi_tri = {
    'Dev': 'Python Developer', 'Python Dev': 'Python Developer', 
    'Data Analyst': 'Data Analyst'
}
df_luong['position'] = df_luong['position'].map(tu_dien_vi_tri).fillna(df_luong['position'])

# === 1. TÍNH ĐIỂM TRUNG BÌNH THEO VỊ TRÍ ===
# Merge để lấy thông tin Vị trí ghép vào bảng Điểm
df_diem_vitri = pd.merge(df_diem, df_luong[['candidate_id', 'position']], on='candidate_id', how='left')
# Groupby và tính Mean
tb_diem_vitri = df_diem_vitri.groupby('position')['score'].mean().round(2).reset_index()
print("\n>>> BẢNG 1: ĐIỂM TRUNG BÌNH THEO VỊ TRÍ")
print(tb_diem_vitri)

# === 2. TÍNH LƯƠNG ĐỀ NGHỊ TRUNG BÌNH THEO VỊ TRÍ ===
tb_luong_vitri = df_luong.groupby('position')['offer_salary'].mean().reset_index()
tb_luong_vitri['Formatted_Salary'] = tb_luong_vitri['offer_salary'].apply(lambda x: "{:,.0f} VNĐ".format(x)) # Định dạng số tiền
print("\n>>> BẢNG 2: LƯƠNG TRUNG BÌNH THEO VỊ TRÍ")
print(tb_luong_vitri[['position', 'Formatted_Salary']])

# === 3. XÁC ĐỊNH ỨNG VIÊN NỔI BẬT (TOP PERFORMERS) ===
diem_ca_nhan = df_diem.groupby('candidate_id')['score'].mean().reset_index()# Tính điểm TB từng người
# Lọc điểm > 7.5
top_ung_vien = diem_ca_nhan[diem_ca_nhan['score'] > 7.5].copy()
top_ung_vien['score'] = top_ung_vien['score'].round(2)
# Ghép tên vào
top_ung_vien = pd.merge(top_ung_vien, df_ung_vien[['candidate_id', 'full_name']], on='candidate_id', how='left')
print("\n>>> BẢNG 3: DANH SÁCH ỨNG VIÊN NỔI BẬT (> 7.5)")
print(top_ung_vien)

# === 4. PHÂN TÍCH TƯƠNG QUAN (KINH NGHIỆM vs LƯƠNG) ===
df_kn_luong = pd.merge(df_ung_vien, df_luong, on='candidate_id', how='inner')
he_so_tuong_quan = df_kn_luong['experience_years'].corr(df_kn_luong['offer_salary'])
print(f"\n>>> HỆ SỐ TƯƠNG QUAN (Correlation): {he_so_tuong_quan:.4f}")

# === 5. TỔNG HỢP CHI TIẾT TỪNG ỨNG VIÊN ===
tong_hop = df_diem.groupby('candidate_id')['score'].agg(['mean', 'min', 'max', 'count']).reset_index()
tong_hop['mean'] = tong_hop['mean'].round(2)
# Đổi tên cột
tong_hop.columns = ['Mã UV', 'Điểm TB', 'Thấp Nhất', 'Cao Nhất', 'Số Vòng']
# Ghép tên
tong_hop = pd.merge(df_ung_vien[['candidate_id', 'full_name']], tong_hop, left_on='candidate_id', right_on='Mã UV', how='left')
print("\n>>> BẢNG 5: TỔNG HỢP CHI TIẾT ")
print(tong_hop)

# Xuất file báo cáo
tong_hop.to_csv('bao_cao_tong_hop_task3.csv', index=False, encoding='utf-8-sig')


>>> BẢNG 1: ĐIỂM TRUNG BÌNH THEO VỊ TRÍ
           position  score
0      Data Analyst   7.10
1  Python Developer   7.07

>>> BẢNG 2: LƯƠNG TRUNG BÌNH THEO VỊ TRÍ
           position Formatted_Salary
0      Data Analyst   19,230,769 VNĐ
1  Python Developer   17,857,143 VNĐ

>>> BẢNG 3: DANH SÁCH ỨNG VIÊN NỔI BẬT (> 7.5)
  candidate_id  score    full_name
0        UV100   9.00  Hoang Van E
1        UV101   7.67     Le Van C
2        UV107   8.00  Hoang Van E
3        UV109   8.33     Le Van C
4        UV113   7.67  Hoang Van E
5        UV117   8.00     Le Van C
6        UV118   7.67   Tran Thi B
7        UV119   7.67   Pham Thi D
8        UV124   7.67   Tran Thi B

>>> HỆ SỐ TƯƠNG QUAN (Correlation): -0.1632

>>> BẢNG 5: TỔNG HỢP CHI TIẾT 
   candidate_id     full_name  Mã UV  Điểm TB  Thấp Nhất  Cao Nhất  Số Vòng
0         UV100   Hoang Van E  UV100     9.00        9.0       9.0        1
1         UV101      Le Van C  UV101     7.67        7.0       9.0        3
2         UV102      L